# Paraphrase Detection on Twitter

In [1]:
#kamal
import numpy as np
import pandas as pd
import math
import copy
import sys
import sklearn.metrics as skm

#nalla
from itertools import product
from collections import defaultdict
import csv
import nltk
from nltk.corpus import stopwords
import gensim
# from pyemd import emd
from gensim import corpora, models, similarities
from scipy.spatial.distance import euclidean
import pulp

#sowrya
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet as wn
from six import iteritems
from collections import deque

## Reading data
* each tweet is considered as a document
* making a list of all the tweets and finding the idf values

In [2]:
def glove_processing():
    f = open('./glove25.txt','r')
    model = {}
    for line in f:
        line = line.split()
        word = line[0]
        embedding = np.array([float(val) for val in line[1:]])
        model[word] =  embedding
    print(len(model))
    return model

In [3]:
df = pd.read_csv('data/train.data', sep='\t', lineterminator='\n', names=['Topic_Id', 'Topic_Name',  'Sent_1', 'Sent_2', 'Label', 'Sent_1_tag','Sent_2_tag'])
df2 = df[['Sent_1', 'Sent_2', 'Label']]
model = glove_processing()

1193514


## Preprocessing Tweets

* Preprocess the tweets with respect to .... to get a 4D vector

In [4]:
def process_labels(labels):
    '''returns binary labels 0 or 1'''
    new_labels = np.array([])
    for i in labels:
        if type(i) == str and len(i) > 1:
            i = i[1]
        if int(i) >= 3:
            new_labels = np.append(new_labels, int(1))
        else:
            new_labels = np.append(new_labels, int(0))
    return new_labels

In [5]:
train_labels = process_labels(df2.Label)
print(train_labels.shape)

(13063,)


### Cosine Distance for average of sentence vectors

In [6]:
def sent_vec(sent, model):
    ans = np.zeros(25)
    count = 0
    if len(sent) == 0:
        return ans
    for i, word in enumerate(sent.split()):
        if word not in model:
            continue
        else:
            ans += np.array(model[word])
            count += 1
    if count > 0:
        ans = ans / count
    return ans, count


def cosine_dist(vec1, vec2):
    ans = 0
    mod1 = 0
    mod2 = 0
    for i in range(vec1.size):
        ans += vec1[i] * vec2[i]
        mod1 += vec1[i] * vec1[i]
        mod2 += vec2[i] * vec2[i]
    if ans == 0:
        return 0
    return ans / (math.sqrt(mod1) * math.sqrt(mod2))


def cosine_data(df, model):
    arr = np.array([]) #has the cosine distance values for all the tweet pairs
    for i in range(df.Sent_1.size):
        vec1, count1 = sent_vec(df.Sent_1[i], model)
        vec2, count2 = sent_vec(df.Sent_2[i], model)
        ans = cosine_dist(vec1, vec2)
        arr = np.append(arr, ans)
#         if i == 7:
#             print(i, df.Sent_1[i], "sdfsdf", df.Sent_2[i], vec1, vec2, count1, count2, ans)
    return arr

### Word movers distance

In [7]:
singleindexing = lambda m, i, j: m*i+j
unpackindexing = lambda m, k: (k/m, k % m)

def tokens_to_fracdict(tokens):
    cntdict = defaultdict(lambda : 0)
    for token in tokens:
        cntdict[token] += 1
    totalcnt = sum(cntdict.values())
    return {token: float(cnt)/totalcnt for token, cnt in cntdict.items()}

# use PuLP
def word_mover_distance_probspec(first_sent_tokens, second_sent_tokens, wvmodel, lpFile=None):
    all_tokens = list(set(first_sent_tokens+second_sent_tokens))
    wordvecs = {}
    for token in all_tokens:
        try:
            wordvecs[token] = wvmodel[token]
        except KeyError:
            wordvecs[token] = 0

    first_sent_buckets = tokens_to_fracdict(first_sent_tokens)
    second_sent_buckets = tokens_to_fracdict(second_sent_tokens)

    T= pulp.LpVariable.dicts('T_matrix', list(product(all_tokens, all_tokens)), lowBound=0)

    prob = pulp.LpProblem('WMD', sense=pulp.LpMinimize)
    try:
        prob += pulp.lpSum([T[token1, token2]*euclidean(wordvecs[token1], wordvecs[token2])
                        for token1, token2 in product(all_tokens, all_tokens)])
    except KeyError:
        prob+=0
    for token2 in second_sent_buckets:
        prob += pulp.lpSum([T[token1, token2] for token1 in first_sent_buckets])==second_sent_buckets[token2]
    for token1 in first_sent_buckets:
        prob += pulp.lpSum([T[token1, token2] for token2 in second_sent_buckets])==first_sent_buckets[token1]

    if lpFile!=None:
        prob.writeLP(lpFile)
    prob.solve()
    if(prob == None):
        print("yes")
    return prob

def word_mover_distance(first_sent_tokens, second_sent_tokens, wvmodel, lpFile=None):
    prob = word_mover_distance_probspec(first_sent_tokens, second_sent_tokens, wvmodel, lpFile=lpFile)
    return pulp.value(prob.objective)

def arr2_creator(df, model):
    arr2 = np.array([])
    stop_words = stopwords.words('english')
    dist=0
    l = open('./data/train.data','r')
    j = 0
    total = 0
    for i in range(df.Sent_1.size):
        sent1 = df.Sent_1[i].split()
        sent2 = df.Sent_2[i].split()
        sent1 = [w for w in sent1 if w not in stop_words]
        sent2 = [w for w in sent2 if w not in stop_words]
        temp = word_mover_distance(sent1, sent2, model)
        if temp == None:
            temp = total / j
        else:
            total += temp
            j += 1
        arr2 = np.append(arr2, temp)
    return arr2

### Wordnet based noun similarity

In [8]:
def penn_to_wn(tag):
    if tag.startswith('N'):
        return 'n'
    # if tag.startswith('V'):
    #     return 'v'
    # if tag.startswith('J'):
    #     return 'a'
    # if tag.startswith('R'):
    #     return 'r'
    return None

def tagged_to_synset(word, tag):
    wn_tag = penn_to_wn(tag)
    if wn_tag is None:
#         print('here')
        return None
    try:
        return wn.synsets(word, wn_tag)[0]
        print('here')
    except:
#         print(wn.synsets(word, wn_tag))
#         print('here_1')
        return None

def shortest_hypernym_paths(synset):
    # hyp = synset.hypernyms()
    # hyp_1 = synset.instance_hypernyms()
    if synset._name == '*ROOT*':
        return {synset: 0}
    queue = deque([(synset,0)])
    path = {}
    while queue:
        s, depth = queue.popleft()
        if s in path:
            continue
        path[s] = depth
        depth += 1
        queue.extend((hyp, depth) for hyp in s.hypernyms())
        queue.extend((hyp, depth) for hyp in s.instance_hypernyms())
    # simulate_root = True
    # # if simulate_root:
    # #     fake_synset = wn.synset(None)
    # #     fake_synset._name = '*ROOT*'
    # #     path[fake_synset] = max(path.values()) + 1
    return path

def shortest_path_distance(synset_1,synset_2):
    if synset_1 == synset_2:
        return 0
    dist_dict1 = shortest_hypernym_paths(synset_1)
    dist_dict2 = shortest_hypernym_paths(synset_2)
    inf = float('inf')
    path_distance = inf
    for synset, d1 in iteritems(dist_dict1):
        d2 = dist_dict2.get(synset, inf)
        path_distance = min(path_distance, d1 + d2)
    return None if math.isinf(path_distance) else path_distance

def path_similarity(synset_1,synset_2):
    distance = shortest_path_distance(synset_1,synset_2)
    if distance is None or distance < 0:
        return None
    return 1.0/(distance + 1)

def sentence_similarity(sentence1, sentence2):
    sentence1, sentence2 = pos_tag(word_tokenize(sentence1)), pos_tag(word_tokenize(sentence2))
    synsets1 = [tagged_to_synset(*tagged_word) for tagged_word in sentence1]
    synsets1 = [ss for ss in synsets1 if ss]
    synsets2 = [tagged_to_synset(*tagged_word) for tagged_word in sentence2]
    synsets2 = [ss for ss in synsets2 if ss]
    score, count = 0.0, 0
    for syn1 in synsets1:
        arr_simi_score = []
        for syn2 in synsets2:
            simi_score = path_similarity(syn1,syn2)
            if simi_score is not None:
                arr_simi_score.append(simi_score)
        if(len(arr_simi_score) > 0):
            best = max(arr_simi_score)
            score += best
            count += 1
    if count == 0:
        score = 0
    else:
        score /= count
    return score

def arr3_creator(df):
    arr3 = np.array([])
    s_1 = "EJ Manuel the 1st QB to go in this draft"
    s_2 = "if EJ is the 1st QB off the board"
    for i in range(df.Sent_1.size):
#     val = (sentence_similarity(s_2, s_1) + sentence_similarity(s_1, s_2))/2
        val = (sentence_similarity(df.Sent_1[i], df.Sent_2[i]) + sentence_similarity(df.Sent_2[i], df.Sent_1[i]))/2
#         print(df.Sent_1[i], df.Sent_2[i], val)
        arr3 = np.append(arr3, val)
    return arr3

### Creating vectors
* Appending all values for each tweet pair to create a 4D vector 

In [9]:
arr1 = cosine_data(df2, model)
print(arr1.shape)

(13063,)


In [10]:
arr2 = arr2_creator(df2, model)
print(arr2.shape)

(13063,)


In [11]:
arr3 = arr3_creator(df2)
print(arr3.shape)

(13063,)


In [12]:
def create_vectors(arr1, arr2, arr3):
    new_arr = np.vstack((arr1, arr2, arr3))
    return new_arr

In [13]:
train_data = create_vectors(arr1, arr2, arr3) #(3,N) dimensions
print(train_data.shape)
#each column is a vector

(3, 13063)


In [18]:
train_data1 = create_vectors(np.zeros(arr1.shape), arr2, arr3) #(3,N) dimensions

In [19]:
train_data2 = create_vectors(arr1, np.zeros(arr2.shape), arr3) #(3,N) dimensions

In [20]:
train_data3 = create_vectors(arr1, arr2, np.zeros(arr3.shape)) #(3,N) dimensions

In [39]:
train_data4 = create_vectors(arr1, np.zeros(arr2.shape), np.zeros(arr3.shape)) #(3,N) dimensions

In [42]:
train_data5 = create_vectors(np.zeros(arr1.shape), arr2, np.zeros(arr3.shape)) #(3,N) dimensions

In [43]:
train_data6 = create_vectors(np.zeros(arr1.shape), np.zeros(arr2.shape), arr3) #(3,N) dimensions

## Training Models

### Logistic Regression

In [14]:
def loss(model, train_data, train_labels):
    ans = 0
    for i in range(train_labels.size):
#         print(np.dot(model, train_data[:, i]))
        val = sigmoid(np.dot(model, train_data[:, i]))
        ans += (train_labels[i] * np.log(val)) + ((1-train_labels[i]) * np.log(1-val))
    return abs(ans)

def classify(val):
    if val >= 0.5:
        return 1
    else:
        return 0

def sigmoid(val):
    ans = 1 / (1 + np.exp(-val))
    return ans

def loss_gradient(model, train_data, train_labels):
    loss = np.zeros(model.size)
    for j in range(train_labels.size):
#         print(j, train_data[:, j])
        loss = loss + train_data[:, j] * (sigmoid(np.dot(model, train_data[:, j])) - train_labels[j])
#         if loss[1] == None:
#             print (j, train_data[:,j])
    return loss

def log_reg(train_data, train_labels):
    '''returns a trained model'''
    eta = 0.00005
    model = np.random.rand(3)
    curr_loss = loss(model, train_data, train_labels)
    prev_loss = curr_loss - 100
    while abs(curr_loss - prev_loss) > 1:
        print(abs(curr_loss - prev_loss))
#     while loss(model, train_data, train_labels) > 0.1:
        val = loss_gradient(model, train_data, train_labels)
        model -= eta * val
        prev_loss = curr_loss
        curr_loss = loss(model, train_data, train_labels)
    return model

In [15]:
lr_model = log_reg(train_data, train_labels)

100.0
19302.17987802352
1535.636791038457
5.654509099948882
2.547519463608296
2.4022999703756795
2.3372411729033047
2.2760281244181897
2.2168219311670327
2.159501190712035
2.1039879786476376
2.0502092278675264
1.998095538504458
1.9475809354489684
1.8986026760376262
1.8511010679058018
1.8050193014742035
1.7603032897359299
1.7169015192803272
1.6747649090602863
1.6338466778533984
1.5941022193128447
1.5554889845734579
1.5179663713734044
1.481495620196256
1.446039715578081
1.4115632938655835
1.3780325562956932
1.3454151863361403
1.3136802728413386
1.2827982366452488
1.2527407623128965
1.2234807324475696
1.1949921673876815
1.1672501669672783
1.1402308563028782
1.1139113343879217
1.0882696252519963
1.0632846329945096
1.0389360974659212
1.0152045545592046


In [21]:
lr_model1 = log_reg(train_data1, train_labels)

100.0
8685.361839845002
87.62052378273347
7.621568359057164
5.515122088274438
5.22653848520622
5.0239685023198035
4.832782901848077
4.649196070119615
4.4727534271778495
4.303167401743849
4.140167813028711
3.983495032094652
3.8328994783050803
3.688141295921014
3.548990015387062
3.415224217512332
3.286631202206081
3.1630066669658845
3.0441543946799356
2.9298859488308153
2.8200203795713605
2.714383937472121
2.6128097977880316
2.515137792052883
2.421214149860134
2.330891248999251
2.244027373935751
2.1604864829778307
2.080137984038629
2.0028565181037266
1.928521750445725
1.857018170117044
1.788234896736867
1.7220654941183966
1.6584077915440503
1.5971637123757318
1.5382391071607344
1.48154359621185
1.4269904161692466
1.3744962732826025
1.3239812028214146
1.2753684336075821
1.2285842583069098
1.1835579085473
1.1402214358358833
1.0985095959576938
1.0583597398235725
1.019711707112947


In [22]:
lr_model2 = log_reg(train_data2, train_labels)

100.0
450.3062813270135
313.5602971813605
218.33834572209526
152.73701633410565
107.66961565557722
76.64972240124735
55.20166981493094
40.28456849455233
29.842482837924763
22.483978621691676
17.263347152225833
13.534061901124005
10.85126323190434
8.906848992548476
7.486090884925034
6.438459825601967
5.657867268467271
5.069184588596727
4.618978738711121
4.269101292936284
3.992223390685467
3.768707567185629
3.584404481733145
3.429093667162306
3.295375366540611
3.177880015462506
3.07270246796179
2.976995913782048
2.888679689541277
2.806228586729958
2.7285206427977755
2.6547270077453504
2.5842321432628523
2.5165759342289675
2.4514116507361905
2.3884753959209775
2.327563882922732
2.268518259643315
2.2112123292126853
2.155543964406206
2.1014288496189693
2.04879591284498
1.997583993371336
1.947739404402455
1.8992141524731778
1.8519646328613817
1.805950672369363
1.7611348268537768
1.7174818622188468
1.6749583717064525
1.6335324901074273
1.5931736813781754
1.5538525778174517
1.5155408572190936


In [23]:
lr_model3 = log_reg(train_data3, train_labels)

100.0
3858.0261010559834
4.987315346509604
1.968169372436023
1.764448102600909
1.685395327887818
1.6139709338049215
1.5455631362774511
1.4798996683603036
1.4168748961001256
1.3563925168355127
1.2983593723720332
1.2426851178252036
1.1892821565052145
1.1380655932052832
1.0889531880320646
1.0418653076385453


In [44]:
lr_model4 = log_reg(train_data4, train_labels)

100.0
719.0709431657269
556.3384653130361
426.2493302237526
324.74269378010104
246.8029333425784
187.53763173399784
142.6980374864179
108.83088942314316
83.23820439996962
63.86105054863765
49.1490983235708
37.942982880987984
29.377983151567605
22.809057027428025
17.75412519823476
13.851830499274001
10.830332968817856
8.484334745921842
6.658169287722558
5.233333239816602
4.119266711061755
3.2465101919133303
2.56160425100461
2.023271456760085
1.5995454482945206
1.2656027067023388
1.0021180450512475


In [45]:
lr_model5 = log_reg(train_data5, train_labels)

100.0
4807.717838277778
12.806094247069268


In [46]:
lr_model6 = log_reg(train_data6, train_labels)

100.0
130.6148563950137
117.53264886976649
105.59735972069939
94.74912915325149
84.92129408141773
76.04328268296376
68.04302272892346
60.84886505012764
54.39105451231808
48.60279953567078
43.420999934811334
38.786694350188554
34.64528525750211
30.946593431590372
27.6447863788635
24.698217604864112
22.069206360712997
19.72378105914686
17.631403996458175
15.7646904397443
14.099131418193792
12.612826649105045
11.286231758334907
10.101922278554412
9.044375637780831
8.09977146046731
7.255809873631733
6.501547092804685
5.827247298006114
5.224249657307155
4.684849303088413
4.202191033307827
3.7701745635367843
3.3833701953626587
3.036943846886061
2.726590458822102
2.448474890587022
2.1991794776931783
1.9756575202973181
1.7751920396785863
1.595359205010027
1.4339959015123895
1.289170969865154
1.159159693725087
1.0424211631852813


In [16]:
print(lr_model)

[-0.31640635 -0.44107213  0.8651805 ]


# testing

In [24]:
df = pd.read_csv('data/test.data', sep='\t', lineterminator='\n', names=['Topic_Id', 'Topic_Name',  'Sent_1', 'Sent_2', 'Label', 'Sent_1_tag','Sent_2_tag'])
test_pairs = df[['Sent_1', 'Sent_2']]
test_labels = process_labels(df.Label)

In [25]:
test_arr1 = cosine_data(test_pairs, model)
print(test_arr1.shape)

(972,)


In [26]:
test_arr2 = arr2_creator(test_pairs, model)
print(test_arr2.shape)

(972,)


In [27]:
test_arr3 = arr3_creator(test_pairs)
print(test_arr3.shape)

(972,)


In [28]:
test_data = create_vectors(test_arr1, test_arr2, test_arr3)

In [30]:
test_data1 = create_vectors(np.zeros(test_arr1.shape), test_arr2, test_arr3)

In [32]:
test_data2 = create_vectors(test_arr1, np.zeros(test_arr2.shape), test_arr3)

In [33]:
test_data3 = create_vectors(test_arr1, test_arr2, np.zeros(test_arr3.shape))

In [54]:
test_data4 = create_vectors(test_arr1, np.zeros(test_arr2.shape), np.zeros(test_arr3.shape))

In [48]:
test_data5 = create_vectors(np.zeros(test_arr1.shape), test_arr2, np.zeros(test_arr3.shape))

In [50]:
test_data6 = create_vectors(np.zeros(test_arr1.shape), np.zeros(test_arr2.shape), test_arr3)

In [34]:
def testing(model, test_data, test_labels):
    predict = np.array([])
    for i in range(test_labels.size):
        val = classify(sigmoid(np.dot(model, test_data[:, i])))
        predict = np.append(predict, val)
    return predict

In [35]:
predicted_arr = testing(lr_model, test_data, test_labels)
print(skm.classification_report(test_labels, predicted_arr))
print(skm.accuracy_score(test_labels, predicted_arr))

              precision    recall  f1-score   support

         0.0       0.69      1.00      0.82       663
         1.0       0.88      0.05      0.09       309

   micro avg       0.70      0.70      0.70       972
   macro avg       0.79      0.52      0.45       972
weighted avg       0.75      0.70      0.59       972

0.6954732510288066


In [36]:
predicted_arr = testing(lr_model1, test_data1, test_labels)
print(skm.classification_report(test_labels, predicted_arr))
print(skm.accuracy_score(test_labels, predicted_arr))

              precision    recall  f1-score   support

         0.0       0.69      0.99      0.82       663
         1.0       0.80      0.06      0.12       309

   micro avg       0.70      0.70      0.70       972
   macro avg       0.75      0.53      0.47       972
weighted avg       0.73      0.70      0.60       972

0.6975308641975309


In [37]:
predicted_arr = testing(lr_model2, test_data2, test_labels)
print(skm.classification_report(test_labels, predicted_arr))
print(skm.accuracy_score(test_labels, predicted_arr))

              precision    recall  f1-score   support

         0.0       0.69      0.94      0.79       663
         1.0       0.37      0.08      0.13       309

   micro avg       0.66      0.66      0.66       972
   macro avg       0.53      0.51      0.46       972
weighted avg       0.59      0.66      0.58       972

0.6646090534979424


In [38]:
predicted_arr = testing(lr_model3, test_data3, test_labels)
print(skm.classification_report(test_labels, predicted_arr))
print(skm.accuracy_score(test_labels, predicted_arr))

              precision    recall  f1-score   support

         0.0       0.68      1.00      0.81       663
         1.0       1.00      0.01      0.01       309

   micro avg       0.68      0.68      0.68       972
   macro avg       0.84      0.50      0.41       972
weighted avg       0.78      0.68      0.56       972

0.684156378600823


In [55]:
predicted_arr = testing(lr_model4, test_data4, test_labels)
print(skm.classification_report(test_labels, predicted_arr))
print(skm.accuracy_score(test_labels, predicted_arr))

              precision    recall  f1-score   support

         0.0       0.69      0.94      0.79       663
         1.0       0.37      0.07      0.12       309

   micro avg       0.66      0.66      0.66       972
   macro avg       0.53      0.51      0.46       972
weighted avg       0.58      0.66      0.58       972

0.6646090534979424


In [52]:
predicted_arr = testing(lr_model5, test_data5, test_labels)
print(skm.classification_report(test_labels, predicted_arr))
print(skm.accuracy_score(test_labels, predicted_arr))

              precision    recall  f1-score   support

         0.0       0.68      1.00      0.81       663
         1.0       1.00      0.00      0.01       309

   micro avg       0.68      0.68      0.68       972
   macro avg       0.84      0.50      0.41       972
weighted avg       0.78      0.68      0.56       972

0.6831275720164609


In [53]:
predicted_arr = testing(lr_model6, test_data6, test_labels)
print(skm.classification_report(test_labels, predicted_arr))
print(skm.accuracy_score(test_labels, predicted_arr))

              precision    recall  f1-score   support

         0.0       0.67      0.86      0.75       663
         1.0       0.21      0.08      0.11       309

   micro avg       0.61      0.61      0.61       972
   macro avg       0.44      0.47      0.43       972
weighted avg       0.52      0.61      0.55       972

0.6141975308641975
